# Tensorflow の script mode での学習と推論を SageMaker で行う

#### ノートブックに含まれる内容

- Tensorflow のscript mode での学習と推論を SageMaker で行うやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: MNIST
- データ: CNN

# セットアップ

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

## 学習データの準備

npy 形式の MNIST データセットが，すでにサンプルデータとして用意されているので，これを使用します．
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [2]:
training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

## 学習用スクリプトの確認

Amazon SageMaker の Script Mode では、トレーニングに用いるコードが実行時に Python スクリプトとして実行されます。その際、データ・モデルの入出力は [こちら](https://sagemaker.readthedocs.io/en/stable/using_tf.html#preparing-a-script-mode-training-script) に記述があるよう SM_CHANNEL_XXXX や SM_MODEL_DIR という環境変数を参照する必要があります。そのため、argparse.ArgumentParser で渡された環境変数と、スクリプト実行時のハイパーパラメータを取得します。

![data-model](./sagemaker-data-model.png)

In [3]:
!pygmentize 'mnist.py'

# Copyright 2018-2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import os
import json
import argparse
from tensorflow.python.platform import tf_logging
import logging as _logging
import sys as _sys


def cnn_model_fn(features, labels, mode):
    """Model functio

## モデルの学習を実行

Tensorflow クラスのオブジェクトを作って，`fit()` で学習を実施します．分散学習を行うためには，まずスクリプト側を分散学習に対応した形式に変える必要があります．

さらに Tensorflow クラスのインスタンス初期化の際に，`train_instance_count` を 2 以上の値にしてください．また分散学習のやり方は，通常の Tensorflow の Parameter server を使用するやり方と，Horovod を使用するやり方の 2 通りを選択可能です．それぞれ `distributions` で下記のような指定をしてください．より詳細な説明については，[こちら](https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training)を参照してください．ここでは， `parameter server` を使用する形でスクリプトを実装しています．Horovod による実装方法は，[Horovod のチュートリアル](https://github.com/horovod/horovod#usage)や[サンプルノートブック](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk/tensorflow_script_mode_horovod)を参照してください．

#### Parameter server

```python
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='tf-train.py', role='SageMakerRole',
                          train_instance_count=2, train_instance_type='ml.p2.xlarge',
                          framework_version='1.11', py_version='py3',
                          distributions={'parameter_server': {'enabled': True}})
tf_estimator.fit('s3://bucket/path/to/training/data')
```

#### Horovod

```python
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='tf-train.py', role='SageMakerRole',
                          train_instance_count=1, train_instance_type='ml.p2.xlarge',
                          framework_version='1.12', py_version='py3',
                          distributions={
                              'mpi': {
                                  'enabled': True,
                                  'processes_per_host': 2,
                                  'custom_mpi_options': '--NCCL_DEBUG INFO'
                              }
                          })
```

In [6]:
from sagemaker.tensorflow import TensorFlow


mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             train_instance_count=2,
                             train_instance_type='ml.m4.xlarge',
                             framework_version='1.13',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

In [7]:
mnist_estimator.fit(training_data_uri)

2019-07-23 05:59:00 Starting - Starting the training job...
2019-07-23 05:59:01 Starting - Launching requested ML instances......
2019-07-23 06:00:05 Starting - Preparing the instances for training......
2019-07-23 06:01:23 Downloading - Downloading input data...
2019-07-23 06:01:53 Training - Training image download completed. Training in progress..
2019-07-23 06:01:56,142 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-07-23 06:01:56,149 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-07-23 06:01:56,492 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-07-23 06:01:56,493 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2019-07-23 06:01:56,493 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-07-23 06:01:56,594 sagemaker_tensorflow_container.training INF

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
2019-07-23 06:02:10,930 sagemaker-containers INFO     Reporting training SUCCESS

2019-07-23 06:03:30 Uploading - Uploading generated training model
2019-07-23 06:03:30 Completed - Training job completed
2019-07-23 06:03:22,731 sagemaker_tensorflow_container.training INFO     master algo-1 is down, stopping parameter server
2019-07-23 06:03:22,731 sagemaker_tensorflow_container.training WARNING  No model artifact is saved under path /opt/ml/model. Your training job will not save any model files to S3.
For details of how to construct your training script see:
https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#adapting-your-local-tensorflow-script
2019-07-23 06:03:22,731 sagemaker-containers INFO     Reporting training SUCCESS
Billable seconds: 255


## モデルの推論を実行

推論を行うために，まず学習したモデルをデプロイします．`deploy()` メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．モデルのデプロイには 10 分程度時間がかかります．script mode では，内部的に Tensorflow Serving を用いているため，saved_model() で出力したモデルは，デプロイ時にそのまま Tensorflow Serving 経由でロードされます．詳細については[こちら](https://sagemaker.readthedocs.io/en/stable/using_tf.html#deploying-tensorflow-serving-models)をご覧ください．

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

推論 API のインプット，アウトプットフォーマットは共に [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest) の predict メソッドと同様です．さらに SageMaker では，それ以外の入力フォーマットにも対応しています．例えば json や jsonlines，csv データなどです．このモデルでは numpy 配列の形式で入力を指定しています．より詳細な推論のサンプルは[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint)をご覧ください．

In [ ]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = predictions['predictions'][i]['classes']
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)